In [4]:
from operator import index
from grpc import Status
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [14]:

"""Email : accessgoogleapi@airbusproject.iam.gserviceaccount.com"""

def getSheet(File,Sheet):
    scopes =['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('secret_key.json',scopes=scopes)
    file = gspread.authorize(creds)
    workbook = file.open(File)
    sheet = workbook.worksheet(Sheet)
    data = sheet.get_all_values()
    df = pd.DataFrame(data[5:],columns=data[3])
    return df

def getItem(df,AM):
    ind = 0
    for i in df.loc[:,"AM"]:
        if i == str(AM) :
            return df.loc[ind]
        ind = ind + 1  

def getStatus(row):
    dic = {'Msg' : 'Votre demande est pris en considération','status' : row['Status'],'Flow': row['Flow']}
    m = 0
    if(str.lower(row['Supplier']) == 'inhouse'):
        m = 'FAUF prod'
    else :
        m = 'PO'
    if(row['Status'] == 'New item') :
        return dic
        
    dic[m] = row[m]
    if (row['Status'] == 'to be checked') :        
        dic['Msg'] = 'Votre demande est en cours de traitement'
        dic[m] = row[m]
        return dic
    if (row['Status'] == 'ordered') :
        dic['Msg'] = 'Votre demande est en cours de traitement'
        dic[m] = row[m]
        dic['ODD'] = row['ODD']
        return dic
    
    dic['DN'] = row['DN']
    dic['QTY'] = row['QTY']
    dic['Prep Date'] = row['Prep Date']

    if (row['Status'] == 'Shipping in progress') :
        dic['Msg'] = 'Votre demande est en cours d’expédiée'
        return dic

    if(row['Status'] == 'shipped') :
        dic['Msg'] = 'Votre demande est expédiée'

    if(row['Status'] == 'Delivered'):
        dic['Msg'] = 'votre demande est délivrée'
    
    dic['Ship Date'] = row['Ship Date']
    
    return dic

    
def getFile(AM):
    files = ['A330 TLS CABIN- Secondary Flow',
            'TLS_A330_Blocking point MP - PODA3 (Potential)',
            'A330 TLS GREEN- Bons_Refus_2020 (Secondary Flow)']
    for file in files :
        df = getSheet(file,'sheet1')
        for i in df.loc[:,"AM"]:
            if i == str(AM) :
                return file
    return 'None'


def main(AM) :
    file = getFile(AM)
    Result = str()
    try :
        #print(file)
        data = getSheet(file,'sheet1')
        row = getItem(data,AM)
        dic = getStatus(row)
        for cle, value in dic.items() :
            Result +=  '\n' + '-' + str(cle) + ' : ' + str(value) 
    except Exception :
        Result = "Votre Demande pas Dispo"
        
    return Result


def getLinkActivitie(num):
    link = 'Numero du lien n\'est pas correcte'
    if num == 1 :
        link = 'https://docs.google.com/document/d/1BgfvIXofvE4HaXtuT7S40bKaAfsRCuDf/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true' 
    if num == 2 :
        link = 'https://docs.google.com/document/d/1hz1xJal5ILCl1vboNzPNThBla0eLAkm2/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true'
    if num == 3 :
        link = 'https://docs.google.com/document/d/1qWjLoEYVC0bJze7KSNe7zjlvdBNCM7IZ/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true'
    if  num == 4 :
        link = 'https://drive.google.com/file/d/1lDP-sYLcKb0-b83urAdSPgw9ys3xKEbn/view?usp=sharing'
    if num == 5 :
        link = 'https://docs.google.com/document/d/1SQur2aveYLYlG-D35ZU5o1Z1DXgAHk1H/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true'
    if num == 6 :
        link ='https://drive.google.com/file/d/1lDP-sYLcKb0-b83urAdSPgw9ys3xKEbn/view?usp=sharing'
    
    return link

def SOI(ints, intents_json,msg):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            if tag == 'Creat WO'  :
                link = 'https://docs.google.com/document/d/1pw7U0eyObx_ByWIhx3St4gLrbm2BsDBv/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true'
            if tag == 'Creat DN' :
                link = 'https://docs.google.com/document/d/1E4mAIN_FAysYmsnnVkysBCr7eoPgMXV0/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true'
            if tag == 'contact SO' :
                link = 'https://docs.google.com/document/d/1RyW7ZfmMIo8bOSBUb1zUJEgWBhrSa6wI/edit?usp=sharing&ouid=101457582345898735112&rtpof=true&sd=true'
            




In [16]:
#print(getLinkActivitie(int('4')))
data = getSheet('test','Feuille 1')
print(data.columns)

Index(['Ordering officier Tlse', 'Item', 'Item \n Received \n OLLS on', 'A/C',
       'MSN\n Cannibalized', 'Type', 'AM\n BDP\n LR\n number', 'PARTNUMBER',
       'ID-Number', 'DESCRIPTION', 'QTY', 'Unit-\n QTY', 'Root Cause',
       'Comment', 'AOG\n yes/\n no', 'deadline -\n ODD TLS', 'Status',
       'Item created \n on SAP PDA', '24.08.2022', 'PO\n / FAUF', 'Remark',
       'Postponed', 'Delivery \n Note \n Number', 'Qty',
       'Shipped \n date \n HAM', 'Prep.', 'A14 Number\n FAUF', 'Supplier',
       'Dispo', 'SO', 'Flow',
       'Item Item Type AM\n BDP\n LR\n number ACA/C HA'],
      dtype='object')
